* NAME: Rishabh Patil
* SAP: 60009200056
* BATCH: D12

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# EXCERCISE 1 - ENGLISH NEWS

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [ ]:
df=df.dropna()

In [ ]:
y=df['label']
X=df.drop('label',axis=1)

In [ ]:
print(X.shape, y.shape)

(18285, 4) (18285,)


In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

2.13.0


In [ ]:
voc_size=5000
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

In [ ]:
sent_length=15
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)
embedded_docs[0]

[[   0    0    0 ... 2527 1955  607]
 [   0    0    0 ... 2905 2849 2753]
 [   0    0    0 ... 3243  221  395]
 ...
 [   0    0    0 ...  544  160 4136]
 [   0    0    0 ... 1848 2886 3321]
 [   0    0    0 ... 2075 4581 1299]]


array([   0,    0,    0,    0,    0, 1176, 3892, 4420, 2757, 3624, 4479,
       4588, 2527, 1955,  607], dtype=int32)

In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:

X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 15), (18285,))

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

In [ ]:
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 100)           500000    
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 580501 (2.21 MB)
Trainable params: 580501 (2.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#Model training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
458/458 [==============================] - 19s 34ms/step - loss: 0.2365 - accuracy: 0.8955 - val_loss: 0.2039 - val_accuracy: 0.9114
Epoch 2/10
458/458 [==============================] - 13s 28ms/step - loss: 0.1242 - accuracy: 0.9532 - val_loss: 0.2080 - val_accuracy: 0.9161
Epoch 3/10
458/458 [==============================] - 12s 27ms/step - loss: 0.0788 - accuracy: 0.9715 - val_loss: 0.2371 - val_accuracy: 0.9128
Epoch 4/10
458/458 [==============================] - 13s 28ms/step - loss: 0.0480 - accuracy: 0.9835 - val_loss: 0.3042 - val_accuracy: 0.9125
Epoch 5/10
458/458 [==============================] - 13s 27ms/step - loss: 0.0251 - accuracy: 0.9916 - val_loss: 0.3718 - val_accuracy: 0.9117
Epoch 6/10
458/458 [==============================] - 13s 28ms/step - loss: 0.0109 - accuracy: 0.9967 - val_loss: 0.5775 - val_accuracy: 0.9136
Epoch 7/10
458/458 [==============================] - 13s 27ms/step - loss: 0.0106 - accuracy: 0.9964 - val_loss: 0.4773 - val_accuracy:

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
predict_x=model.predict(X_test)
print(predict_x)
y_pred=np.argmax(predict_x,axis=1)

115/115 [==============================] - 1s 6ms/step
[[9.9999869e-01]
 [5.3865366e-05]
 [5.2893887e-05]
 ...
 [9.9955797e-01]
 [5.4667819e-08]
 [9.9931222e-01]]


In [ ]:
confusion_matrix(y_test,y_pred)

array([[2082,    0],
       [1575,    0]])

In [ ]:
accuracy_score(y_test,y_pred)

0.5693191140278917

# EXERCISE 2 - HINDI NEWS

In [ ]:
# Load the CSV files
fake_news_df = pd.read_csv('/content/drive/MyDrive/fake_news_basic.csv')
true_news_df = pd.read_csv('/content/drive/MyDrive/fake_news_basic.csv')

# Add labels to indicate the class (0 for true news, 1 for fake news)
fake_news_df['label'] = 1
true_news_df['label'] = 0

# Concatenate the DataFrames
combined_dataset = pd.concat([fake_news_df, true_news_df], ignore_index=True)

# Shuffle the combined dataset
combined_dataset = combined_dataset.sample(frac=1).reset_index(drop=True)

# Save the combined dataset to a new CSV file
combined_dataset.to_csv('combined_news_dataset.csv', index=False)

In [ ]:
# Read the CSV file into a Pandas DataFrame
combined_news_df = pd.read_csv('combined_news_dataset.csv')
df=combined_news_df

In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
column_names = df.columns

# Now, 'column_names' is a list containing the names of columns in your DataFrame
print(column_names)

Index(['Unnamed: 0', 'short_description', 'label'], dtype='object')


In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
X.shape
print(X)

      Unnamed: 0                                  short_description
0            955  इंडियन यूनियन मुस्लिम लीग के झंड पाकिस्तान झंड...
1            512  बूम पाय यूनिसेफ़ ऐस एडवाइज़र जार नह मैसेज ज्यादा...
2            333  बूम पाय तस्वीर हैंडओवर समारोह दक्षिण कोरिय कोर...
3           1001  सोनिय गांध मालदीव्स के पूर्व राष्ट्रपत फ़ोट किय...
4            935  जेएनय छात्र नजीब अहमद 15 अक्टूबर 2016 लाप गय त...
...          ...                                                ...
2495        1208  पाकिस्ता पत्रकार हामिद मीर वीडिय साझ कश्मीर भा...
2496         303  बूम पाय तस्वीर 2012 इंटरनेट मौजूद रशियन वेबसाइ...
2497         757  बूम मेरठ पुलिस बात बत मुज़फ़्फ़रनगर गिरोह काफ दिन...
2498         472  बूम पत लग वीडिय नवंबर 2015 योग आदित्यनाथ असहिष...
2499         492  बूम पाय वायरल ऑडिय चेन्नई के ऑर्थोपेडिशियन डॉ ...

[2500 rows x 2 columns]


In [ ]:
y.shape
print(y)

0       1
1       1
2       1
3       1
4       0
       ..
2495    0
2496    0
2497    1
2498    1
2499    1
Name: label, Length: 2500, dtype: int64


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
#Onehot Representation

messages=X.copy()

In [ ]:
messages['short_description'][2]

'बूम पाय तस्वीर हैंडओवर समारोह दक्षिण कोरिय कोरिय युद्ध लड़ वाल ची सैनिक के '

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
!pip install indic-nlp-library


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
import nltk
import string
from nltk.corpus import stopwords
from indicnlp.tokenize import sentence_tokenize, indic_tokenize

In [ ]:
def preprocess_text(text):
    # Tokenization - Using Indic NLP for tokenization
    sentences = sentence_tokenize.sentence_split(text, lang='hi')
    tokens = [indic_tokenize.trivial_tokenize(sentence) for sentence in sentences]
    # Flatten the list of tokens
    tokens = [token for sublist in tokens for token in sublist]

    # Remove punctuation and numbers
    tokens = [token for token in tokens if token not in string.punctuation and not token.isdigit()]

    # Remove stopwords
    stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']
    stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    #punctuations = ['nn','n', '।','/', '`', '+', '\', '"', '?', '▁(', '$', '@', '[', '_', "'", '!', ',', ':', '^', '|', ']', '=', '%', '&', '.', ')', '(', '#', '*', '', ';', '-', '}','|','"']
    stop_words = stopwords_hi + stopwords_en
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization or stemming (you can choose one)
    # For stemming, you can use SnowballStemmer from nltk.stem.snowball
    # For lemmatization, you may need a specialized library like spaCy with a Hindi language model
    # Example for stemming:
    # stemmer = SnowballStemmer("hindi")
    # tokens = [stemmer.stem(token) for token in tokens]

    # Join tokens back into a cleaned text
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Apply preprocessing to each text in your dataset
df['cleaned_text'] = df['short_description'].apply(preprocess_text)




In [ ]:

corpus = df['cleaned_text']

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

In [ ]:
#Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3080 3794 4245]
 [   0    0    0 ... 2783  606 1556]
 [   0    0    0 ... 4024 2881 1010]
 ...
 [   0    0    0 ... 4693 1601 4749]
 [   0    0    0 ... 3026 2786  910]
 [   0    0    0 ... 2757  437 1556]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0, 2963, 1371, 2869,
       1756,  612, 4663,  612, 3640, 4284, 3080, 3794, 4245], dtype=int32)

In [ ]:
### Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(2500, (2500,))

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((2500, 20), (2500,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.15, random_state=42)

In [ ]:
#Model training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
34/34 [==============================] - 5s 52ms/step - loss: 0.6941 - accuracy: 0.4786 - val_loss: 0.6940 - val_accuracy: 0.4827
Epoch 2/20
34/34 [==============================] - 1s 37ms/step - loss: 0.6936 - accuracy: 0.4918 - val_loss: 0.6948 - val_accuracy: 0.4827
Epoch 3/20
34/34 [==============================] - 1s 38ms/step - loss: 0.6933 - accuracy: 0.5021 - val_loss: 0.6963 - val_accuracy: 0.4053
Epoch 4/20
34/34 [==============================] - 3s 77ms/step - loss: 0.6932 - accuracy: 0.5125 - val_loss: 0.6998 - val_accuracy: 0.2400
Epoch 5/20
34/34 [==============================] - 3s 87ms/step - loss: 0.6928 - accuracy: 0.5111 - val_loss: 0.7073 - val_accuracy: 0.1307
Epoch 6/20
34/34 [==============================] - 2s 61ms/step - loss: 0.6926 - accuracy: 0.4969 - val_loss: 0.7299 - val_accuracy: 0.1067
Epoch 7/20
34/34 [==============================] - 1s 35ms/step - loss: 0.6911 - accuracy: 0.4993 - val_loss: 0.7583 - val_accuracy: 0.1173
Epoch 8/20
34

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#y_pred=model.predict_classes(X_test)
predict_x=model.predict(X_test)
y_pred=np.argmax(predict_x,axis=1)

12/12 [==============================] - 1s 7ms/step


In [ ]:
confusion_matrix(y_test,y_pred)

array([[181,   0],
       [194,   0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4826666666666667